In [ ]:
import spacy
import csv

In [ ]:
def divide_text_into_sentences(file_path):
    # load English tokenizer, POS tagger, parser and NER
    nlp = spacy.load("en_core_web_sm")

    with open(file_path, 'r') as f:
        text = f.read()

    # process the text
    doc = nlp(text)

    # write sentences to a CSV file
    with open('Data.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for sentence in doc.sents:
            writer.writerow([sentence.text])


divide_text_into_sentences('RawText.txt')

In [ ]:
import csv
import openai
import json

In [ ]:
openai.api_key = ''

In [ ]:
def extract_information(file_path):
    with open(file_path, 'r') as f:
        text = f.read()
    return text


# def generate_triples(text):
#     response = openai.Completion.create(
#         # engine="gpt-3.5-turbo",
#         engine="text-davinci-003",
#         prompt=f"Generate All the RDF triples for the information below in URI format by:'{text}'. Write every triple in a independent line please and you are allowed to adjust all the triples to use URLs from original sources. For Example:  ('<https://en.wikipedia.org/wiki/Acute_lymphoblastic_leukemia>', '<https://en.wikipedia.org/wiki/Property_(philosophy)>/typeOf>', '<https://en.wikipedia.org/wiki/Cancer>').",
#         temperature=0.5,
#         max_tokens=3699
#     )
#     triple_text = response.choices[0].text.strip()
#     return triple_text

def generate_triples(text):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Generate All the RDF triples in URI format you can found from the information below :'{text}'. Write every URI triple in an independent line as JSON. http://dbpedia.org/ontology is allowed for use,Subject is either a URI or a blank node.Predicate is always a URI. Object is either a URI, a blank node or a literal.",
        temperature=0.6,
        max_tokens=2000  # be careful with max tokens value. Too high values may result in API errors.
    )
    triple_text = response.choices[0].text.strip()
    print(f"Generated triples: {triple_text}")
    return triple_text

def output_to_json(triples, output_path):
    with open(output_path, 'w') as f:
        json.dump(triples, f)


# def main(input_path, output_path):
#     text = extract_information(input_path)
#     triples = generate_triples(text)
#     output_to_json(triples, output_path)

def main(input_path, output_path):
    triples = []
    with open(input_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            triple = generate_triples(row[0])
            triples.append(triple)
    output_to_json(triples, output_path)

main('Data.csv', 'UDITriples.json')


In [ ]:
import json
from py2neo import Graph, Node, Relationship

In [ ]:
# connect to your Neo4j instance
graph = Graph("http://127.0.0.1:7474", user="neo4j", password="***")

In [ ]:
def categorize_node(uri):
    if 'Disease' in uri:
        return 'Disease'
    elif 'Symptom' in uri:
        return 'Symptom'
    elif 'Treatment' in uri:
        return 'Treatment'
    elif 'Cell' in uri:
        return 'Cell'
    elif 'Examination' in uri:
        return 'Examination'
    else:
        return 'Resource'


# process each triple
for triple_data in triples_data:
    triple = json.loads(triple_data)

    # retrieve the subject, predicate, and object
    subject_uri = triple.get('Subject') or triple.get('subject')
    predicate_uri = triple.get('Predicate') or triple.get('predicate')
    object_uri = triple.get('Object') or triple.get('object')

    # log information
    print(f'Processing triple: {subject_uri}, {predicate_uri}, {object_uri}')

    # determine the category of the subject and object
    subject_category = categorize_node(subject_uri)
    object_category = categorize_node(object_uri)

    # create nodes and relationship
    subject = Node(subject_category, uri=subject_uri)
    object = Node(object_category, uri=object_uri)
    predicate = Relationship.type(predicate_uri)

    # check if nodes already exist in the graph, if not create them
    graph.merge(subject, subject_category, "uri")
    graph.merge(object, object_category, "uri")

    # create the relationship
    rel = predicate(subject, object)
    graph.merge(rel)

    # print logging message
    print(f"Created nodes: {subject_uri} ({subject_category}), {object_uri} ({object_category})")
    print(f"Created relationship: {subject_uri} -[{predicate_uri}]-> {object_uri}")